In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_21047/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_21047/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_29_4,0.999583,0.826251,0.817434,0.999269,0.001737,1.161863,0.653027,0.002464,0.111679,0.041682,1.026712,0.042338,142.710769,284.045942,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_29_6,0.999595,0.826179,0.817536,0.999287,0.001687,1.162342,0.652662,0.002401,0.111229,0.041073,1.025938,0.041720,142.769607,284.104780,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_29_0,0.999523,0.826083,0.817296,0.999288,0.001988,1.162981,0.653523,0.002399,0.119511,0.044582,1.030560,0.045284,142.441680,283.776853,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_30_7,0.998969,0.826024,0.811777,0.995892,0.004290,1.163380,0.673262,0.020525,0.096355,0.065495,1.065953,0.066526,140.903137,282.238310,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_31_0,0.998850,0.825955,0.811047,0.995008,0.004788,1.163839,0.675872,0.008369,0.093850,0.069194,1.073614,0.070284,140.683370,282.018542,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,model_1_3_7,0.924682,0.734786,0.764976,0.908051,0.313513,1.773484,0.840666,0.247903,0.538213,0.559923,5.820356,0.568741,132.319826,273.654999,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
139,model_1_3_3,0.908184,0.721755,0.752498,0.887204,0.382186,1.860627,0.885299,0.304107,0.639623,0.618212,6.876214,0.627948,131.923696,273.258869,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
140,model_1_3_0,0.902906,0.717329,0.748450,0.881182,0.404155,1.890220,0.899781,0.320344,0.638576,0.635732,7.213989,0.645744,131.811916,273.147088,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
141,model_1_2_1,0.883463,0.702596,0.741144,0.853835,0.485089,1.988742,0.925914,0.715807,0.737690,0.696483,8.458373,0.707452,131.446847,272.782019,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
